---
**NOTE**

You can convert this document to an .R or .Rmd file if you prefer by using the python library [`jupytext`](https://github.com/mwouts/jupytext).
To do so, type the following command:


```bash
jupytext --to Rmd create_hbadeals_metadata.ipynb #or
jupytext --to R   create_hbadeals_metadata.ipynb
```
You should see something like the following in your console after typing the above command:

```
[jupytext] Reading create_hbadeals_metadata.ipynb
[jupytext] Writing create_hbadeals_metadata.R
```

The above tells you that the `.R` version of the `.ipynb` notebook has been created. 

---

# Creating the required input files for running [TheJacksonLaboratory/coronavirus-nf](https://github.com/TheJacksonLaboratory/coronavirus-nf/blob/master/main.nf#L1645-L1673)


The helper script was created to create the required metadata files for the parameters:


```bash
--accessionList
--hbadeals_metadata
```

```console
Mandatory arguments:
      --accessionList [file]        Path to input file with accession list to fetch from SRA.
                                    The file should be a one column list with no header.
                                    
      --hbadeals_metadata [file]    Path to input master file that contains the metadata for the contrasts.
                                    The file should be a two column .csv file with a header 
                                    (eg colnames: "contrast", "metadata_file").
                                    
                                    The 1st column must have a unique id that describes the cohort.
                                    The 2nd column must point to the path of the cohort metadata file.
                                    
                                    The metadata file must have the following information:
                                    - SRRxx id
                                    - status
                                    This will be used as input for the rsem2hbadeals.R script.
                                    See rsem2hbadeals.R --help for specifications of this files.
                                    
```

# Loading libraries

To install the following libraries if you are in a JupyterLab Notebook Session, open a terminal from `File > New > Terminal` and install via conda by typing the following command:


```bash
conda install r-dplyr r-rlist r-snakecase -y
```

In [35]:
suppressWarnings(suppressMessages(library(dplyr)))
suppressWarnings(suppressMessages(library(rlist)))
suppressWarnings(suppressMessages(library(snakecase)))

# Generating required files from the [processDatasets.py](https://github.com/TheJacksonLaboratory/coronavirus-sample-pre-processing-nf/blob/master/scripts/processDatasets.py)


**Note**: Using the latest one by date, [`case_control-april21.tsv`](https://github.com/TheJacksonLaboratory/coronavirus-sample-pre-processing-nf/tree/655ccc8aead83ecd40168fa83825f2f87c3cf28a/data)

The following code won't work unless you have the token, to get the link with the token you have to be logged in to GitHub in the browser that you are opening the following link. Copy the url of the file here [`case_control-april21.tsv`](https://github.com/TheJacksonLaboratory/coronavirus-sample-pre-processing-nf/tree/655ccc8aead83ecd40168fa83825f2f87c3cf28a/data)

In [38]:
metadata <- as.data.frame(data.table::fread("https://raw.githubusercontent.com/TheJacksonLaboratory/coronavirus-sample-pre-processing-nf/655ccc8aead83ecd40168fa83825f2f87c3cf28a/data/case_control-april21.tsv?token=NOTTHEREALTOKEN"))

In [39]:
head(metadata,2)

,cohort,srp,srr,category,status
,<int>,<chr>,<chr>,<chr>,<chr>
1,1,SRP230823,SRR10503939,H5N1.infected,case
2,1,SRP230823,SRR10503924,mock,control


# Step 1. Create an individual dataframe for each contrast

And store all the individual cohort dataframes in a list to be able in the next step to write them into seperate .csv files.<br>
Below I am printing a summary of the list to check I have all **49** dataframes, one for each contrast.

In [21]:
contrasts_ids       <- unique(metadata$cohort)

all_contrasts_meta <- list()
for (contrast_id in contrasts_ids){
  contrast_meta <- metadata[metadata$cohort == contrast_id , ]
  all_contrasts_meta <- rlist::list.append(all_contrasts_meta, contrast_meta)
}

head(summary(all_contrasts_meta))
length(all_contrasts_meta)

     Length Class      Mode
[1,] 5      data.frame list
[2,] 5      data.frame list
[3,] 5      data.frame list
[4,] 5      data.frame list
[5,] 5      data.frame list
[6,] 5      data.frame list

[1] 49

# Step 2. Loop over every contrast metadata dataframe and write as .csv

We will also use some of the columns to create a unique id that describes each contrast. I chose here to add the **`srp`** and **`category`** values of the `case` subjects, as it might provide more information that the names of the controls. <br>


Briefly, the following block will:

1. access each of the contrast dataframes based on their position in the list named **`all_contrasts_meta`**
2. assign the **i-th** element of the list (**contrast dataframe i**) to the variable `meta`
3. re-order the **`meta`** dataframe based on column 'status' (a case will be the first row)
4. save in the **`name`** variable the unique id derived from **`srp`**, **`category`** of a case dataframe entry. $^{*}$
5. extract info on total **sample size of cohort**, number of controls and number of cases in each contrast
6. keep only relevant columns initially named **`srr`** **`status`**, and rename **`srr`** to **`sample_id`** 
7. based on the info retrieved from 4., 5. create a filename that describes the contrast and adds info on sample size
8. write each contrast metadata modified dataframe as a .csv file with 2 columns and a header (colnames: **`sample_id`** ,**`status`** ).

\* **NOTE:**  I am using also the function `snakecase::to_parsed_case()` to clean characters like `/` in the filename that might break filepaths.

In [31]:
savedir <- '../data/'

for (i in seq_along(all_contrasts_meta)) {
  meta <- all_contrasts_meta[[i]]
  cohort <- meta$cohort[1]
  meta <- meta[order(meta$status),]
  name <-  snakecase::to_parsed_case(paste(meta[1, c('srp', 'category')], collapse = "_"))
  N_cohort   <- dim(meta)[1]
  N_controls <- dim(meta[meta$status == 'control', ])[1]
  N_cases    <- dim(meta[meta$status == 'case', ])[1]
  toWrite <- meta [, c("srr","status")]
  colnames(toWrite) <- c("sample_id", "status")
  data.table::fwrite( toWrite, 
                     file = paste0(savedir, 
                                   name, 
                                   "_cohort_", cohort, 
                                   "__Nctrl_", N_controls ,
                                   "__Ncases_", N_cases, 
                                   ".csv") , 
                     sep = ",", col.names = TRUE, row.names = FALSE)
    
  message( N_cohort, ", ", N_controls, ", " ,  N_cases, "          ", name, "  ", appendLF = FALSE)
}

10, 6, 4          SRP_230823_H_5_N_1_infected  
15, 6, 9          SRP_040070_MRC_5_low_MOI_SARS_n_a_24_h  
15, 6, 9          SRP_040070_MRC_5_High_MOI_SARS_n_a_24_h  
11, 6, 5          SRP_040070_MRC_5_low_MOI_SARS_n_a_48_h  
10, 6, 4          SRP_040070_MRC_5_High_MOI_SARS_n_a_48_h  
12, 6, 6          SRP_040070_MRC_5_low_MOI_MERS_n_a_24_h  
12, 6, 6          SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h  
11, 6, 5          SRP_040070_MRC_5_low_MOI_MERS_n_a_48_h  
10, 6, 4          SRP_040070_MRC_5_High_MOI_MERS_n_a_48_h  
13, 9, 4          SRP_040070_MRC_5_SARS_IF_Nbeta_24_h  
9, 5, 4          SRP_040070_MRC_5_SARS_IF_Nbeta_48_h  
13, 9, 4          SRP_040070_MRC_5_SARS_Gleevec_24_h  
9, 5, 4          SRP_040070_MRC_5_SARS_Gleevec_48_h  
14, 6, 8          SRP_040070_MRC_5_MERS_Gleevec_24_h  
13, 5, 8          SRP_040070_MRC_5_MERS_Gleevec_48_h  
14, 6, 8          SRP_040070_MRC_5_MERS_IF_Nbeta_24_h  
13, 5, 8          SRP_040070_MRC_5_MERS_IF_Nbeta_48_h  
4, 2, 2          SRP_076102_flu_A_

## **NOTE:**

We observe above that some of the contrasts have a very small sample size.
I am not excluding those from the CloudOS run, but these will fail, especially the ones with < 10 N, sample size.

# Step 3. Add an extra contrast with all controls vs all cases (any viral infection) and generate the **`--accessionList`** input

To create a contrast with a larger sample size, assuming we do not have biological replicates with different SRR ids, which would inflate falsely our sample size we can keep the unique control and unique cases across any viral infection. This might give more robust results as we will have a larger sample size.

For creating the contrast 'all contrasts vs all cases' see block below:

In [40]:
metadata <- metadata[, c("srr","status")]
colnames(metadata) <- c("sample_id", "status")

In [42]:
head(metadata, 2)
dim(metadata)

,sample_id,status
,<chr>,<chr>
1,SRR10503939,case
2,SRR10503924,control


[1] 537   2

In [43]:
# use the function 'dplyr::distinct()' to keep unique rows
metadata <- dplyr::distinct(metadata)

In [44]:
head(metadata, 2)
dim(metadata)

,sample_id,status
,<chr>,<chr>
1,SRR10503939,case
2,SRR10503924,control


[1] 351   2

We have **351** subjects in our cohort (unique **SRR** ids). <br>
Let's now inspect the breakdown by class, **`control`**, **`case`**.

In [51]:
n_controls <- length(metadata$sample_id[metadata$status == "control"])
n_cases    <- length(metadata$sample_id[metadata$status == "case"])

message("Number of controls: ", n_controls, appendLF = FALSE)
message("Number of cases   : ", n_cases   , appendLF = FALSE) 

Number of controls: 143
Number of cases   : 208


In [75]:
# create the accession list

data.table::fwrite(as.data.frame(metadata$sample_id), 
                   file = paste0(savedir, "accession_N351.list") , 
                   col.names = FALSE, 
                   row.names = FALSE)
cat(list.files(path =  "../data", 
               full.names = TRUE,
               pattern = "*accession_N351.list"))

../data/accession_N351.list

In [76]:
# write in a file the all vs all contrast metadata table
data.table::fwrite( toWrite, file = paste0( savedir, 
                                           "all_unique_controls_vs_all_cases", 
                                           "__Nctrl_", n_controls ,
                                           "__Ncases_", n_cases, 
                                           ".csv") , 
                   sep = ",", col.names = TRUE, row.names = FALSE)

cat(list.files(path =  "../data", 
               full.names = TRUE,
               pattern = "*all_unique_controls_vs_all_cases*"))

../data/all_unique_controls_vs_all_cases__Nctrl_143__Ncases_208.csv

Finally let's list all the contrast metadata files (we expect 50, 49 contrasts + 1 all vs all)

In [63]:
metadata_files <- list.files(path = savedir,pattern = "*__Nctrl_*")
length(metadata_files)

[1] 50

# Step 4. Make the created files contrast metadata files available

We can use any of the following paths with Nextflow:

1. **`my/path/only/I/have/access/to`**           (local not recommended unless in an HPC environment)
2. **`https://`**  
3. **`s3://`**  (AWS)
4. **`gs://`**  (Google Cloud Platform)

I opted for option 2. and for keeping the input data in the same place as the pipeline, I once again opted for the heuristic suggested by the authors of the R package `ropensci/piggyback`. I uploaded the files in a release in the pipeline repository, under the tag name `SRA`.

This can be found here:<br>
https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/tag/SRA

All the files can now be accessible in the url with the pattern:

```console
https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/<file_basename>
```

# Step 5. Use the available links to generate the input for **`--hbadeals_metadata`**


We will now use the list of files that we saved in the variable 'metadata_files' in **Step 3**.

In [66]:
cat(metadata_files[1:3], sep = "\n")

all_unique_controls_vs_all_cases__Nctrl_143__Ncases_208.csv
SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h_cohort_7__Nctrl_6__Ncases_6.csv
SRP_040070_MRC_5_High_MOI_MERS_n_a_48_h_cohort_9__Nctrl_6__Ncases_4.csv


In [67]:
url <- paste0("https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/", metadata_files)

In [68]:
cat(url[1:3], sep = "\n")

https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/all_unique_controls_vs_all_cases__Nctrl_143__Ncases_208.csv
https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h_cohort_7__Nctrl_6__Ncases_6.csv
https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/SRP_040070_MRC_5_High_MOI_MERS_n_a_48_h_cohort_9__Nctrl_6__Ncases_4.csv


In [82]:
metadata <- data.frame(metadata_files, url)
colnames(metadata) <- c("contrast","metadata_file")

In [83]:
head(metadata, 2)

,contrast,metadata_file
,<fct>,<fct>
1,all_unique_controls_vs_all_cases__Nctrl_143__Ncases_208.csv,https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/all_unique_controls_vs_all_cases__Nctrl_143__Ncases_208.csv
2,SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h_cohort_7__Nctrl_6__Ncases_6.csv,https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h_cohort_7__Nctrl_6__Ncases_6.csv


Save to a file and then upload all to the GitHub release:

In [84]:
data.table::fwrite( metadata, 
                   file = paste0( savedir,"sra_masterfile_n351.csv"),
                   sep = ",", 
                   col.names = TRUE,
                   row.names = FALSE)

cat(list.files(path =  "../data", 
               full.names = TRUE,
               pattern = "*sra_masterfile_n351.csv*"))

../data/sra_masterfile_n351.csv

# Step 6: Create a release in the GitHub repo to host the data in the binaries section

The release which has the input data we need for running the pipeline can be found here:<br>
https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/tag/SRA

We will run the Nextflow pipeline with these arguments:

```groovy
nextflow run main.nf \
--accesionList https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/accession_N351.list \
--hbadeals_metadata https://github.com/TheJacksonLaboratory/coronavirus-nf/releases/download/SRA/sra_masterfile_n351.csv \
-- ...etc

```

# Checksums of files with the sha256 algorithm 

In [89]:
message("Generating sha256 checksums of the files in the `..data/` directory .. ")
system("cd ../data/ && sha256sum * > sha256sums.txt", intern = TRUE)
message("Done!\n")

data.table::fread("../data/sha256sums.txt", header = FALSE, col.names = c("sha256sum", "file"))

Generating sha256 checksums of the files in the `..data/` directory .. 



character(0)

Done!




sha256sum,file
<chr>,<chr>
ea1600b9fb33e2e5c582a389df987ed802b9a2290c3491f5db1a3cc906de04a1,SRP_040070_MRC_5_High_MOI_MERS_n_a_24_h_cohort_7__Nctrl_6__Ncases_6.csv
eb58677eff523b64fd1941125f20be0037ecb78cd812d1ddd1ffe1d5f96b1868,SRP_040070_MRC_5_High_MOI_MERS_n_a_48_h_cohort_9__Nctrl_6__Ncases_4.csv
5598074469b3b712ff90ba4dc1f97aae08592fd07620015b8359a626e9402c34,SRP_040070_MRC_5_High_MOI_SARS_n_a_24_h_cohort_3__Nctrl_6__Ncases_9.csv
705212fb2c25d028a6dc19dfe5092152d2e3a7f590bf63964e403f00931da467,SRP_040070_MRC_5_High_MOI_SARS_n_a_48_h_cohort_5__Nctrl_6__Ncases_4.csv
3fa4417422ec578701f52b0afe2657bde06d83c712300e4d359a0d29ce9fe13e,SRP_040070_MRC_5_MERS_Gleevec_24_h_cohort_14__Nctrl_6__Ncases_8.csv
fb37f1aa2ba7d91ed1457a8b9cad39d3682e207fc95d48fe9456c94462534b08,SRP_040070_MRC_5_MERS_Gleevec_48_h_cohort_15__Nctrl_5__Ncases_8.csv
38745691c0d20938ab602b0f4c8c9574bcf94a2166f0c0bbf9eee47e6237960d,SRP_040070_MRC_5_MERS_IF_Nbeta_24_h_cohort_16__Nctrl_6__Ncases_8.csv
341c169aa4d4ce37566db127db3a7159b7e88a17f73587b27034041e2c0de99a,SRP_040070_MRC_5_MERS_IF_Nbeta_48_h_cohort_17__Nctrl_5__Ncases_8.csv
87fddb4ecd9edc43712fd8d47cb0a7b8d6db1487331fa3da483584824abe9782,SRP_040070_MRC_5_SARS_Gleevec_24_h_cohort_12__Nctrl_9__Ncases_4.csv


# Libraries metadata

In [90]:
 utils::sessionInfo()

R version 3.6.3 (2020-02-29)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] dplyr_0.8.5      snakecase_0.11.0 rlist_0.4.6.1   

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.4.6      magrittr_1.5      tidyselect_1.0.0  uuid_0.1-4       
 [5] R6_2.4.1          rlang_0.4.5       fansi_0.4.1       stringr_1.4.0    
 [9] tools_3.6.3       data.table_1.12.8 cli_2.0.2         ellipsis_0.3.